In [40]:
import numpy as np
import pandas as pd
from random import random
import os
import librosa
import soundfile as sf
import IPython
from scipy.linalg import svd
from numpy import load, zeros, ones, asarray, vstack
from numpy.random import randint
import tensorflow as tf
import keras
from keras.optimizers import Adam
from keras.initializers import RandomNormal
from keras.models import Model, load_model
from keras.layers import Conv2D, Conv2DTranspose, LeakyReLU, Activation, Concatenate, Layer, InputSpec, Input, Conv1D, Conv1DTranspose
from keras import initializers, regularizers, constraints, backend as K
from matplotlib import pyplot
from music21 import midi
from midiutil import MIDIFile
import fluidsynth
from pydub import AudioSegment
import IPython.display as ipd
from pretty_midi import PrettyMIDI

### Model definition

In [41]:
# Instance normalizer
# Refrenced from: https://github.com/keras-team/keras-contrib/blob/master/keras_contrib/layers/normalization/instancenormalization.py

class InstanceNormalization(Layer):
    def __init__(self,
                 axis=None,
                 epsilon=1e-3,
                 center=True,
                 scale=True,
                 beta_initializer='zeros',
                 gamma_initializer='ones',
                 beta_regularizer=None,
                 gamma_regularizer=None,
                 beta_constraint=None,
                 gamma_constraint=None,
                 **kwargs):
        super(InstanceNormalization, self).__init__(**kwargs)
        self.supports_masking = True
        self.axis = axis
        self.epsilon = epsilon
        self.center = center
        self.scale = scale
        self.beta_initializer = initializers.get(beta_initializer)
        self.gamma_initializer = initializers.get(gamma_initializer)
        self.beta_regularizer = regularizers.get(beta_regularizer)
        self.gamma_regularizer = regularizers.get(gamma_regularizer)
        self.beta_constraint = constraints.get(beta_constraint)
        self.gamma_constraint = constraints.get(gamma_constraint)

    def build(self, input_shape):
        ndim = len(input_shape)
        if self.axis == 0:
            raise ValueError('Axis cannot be zero')

        if (self.axis is not None) and (ndim == 2):
            raise ValueError('Cannot specify axis for rank 1 tensor')

        self.input_spec = InputSpec(ndim=ndim)

        if self.axis is None:
            shape = (1,)
        else:
            shape = (input_shape[self.axis],)

        if self.scale:
            self.gamma = self.add_weight(shape=shape,
                                         name='gamma',
                                         initializer=self.gamma_initializer,
                                         regularizer=self.gamma_regularizer,
                                         constraint=self.gamma_constraint)
        else:
            self.gamma = None
        if self.center:
            self.beta = self.add_weight(shape=shape,
                                        name='beta',
                                        initializer=self.beta_initializer,
                                        regularizer=self.beta_regularizer,
                                        constraint=self.beta_constraint)
        else:
            self.beta = None
        self.built = True

    def call(self, inputs, training=None):
        input_shape = K.int_shape(inputs)
        reduction_axes = list(range(0, len(input_shape)))

        if self.axis is not None:
            del reduction_axes[self.axis]

        del reduction_axes[0]

        mean = K.mean(inputs, reduction_axes, keepdims=True)
        stddev = K.std(inputs, reduction_axes, keepdims=True) + self.epsilon
        normed = (inputs - mean) / stddev

        broadcast_shape = [1] * len(input_shape)
        if self.axis is not None:
            broadcast_shape[self.axis] = input_shape[self.axis]

        if self.scale:
            broadcast_gamma = K.reshape(self.gamma, broadcast_shape)
            normed = normed * broadcast_gamma
        if self.center:
            broadcast_beta = K.reshape(self.beta, broadcast_shape)
            normed = normed + broadcast_beta
        return normed

    def get_config(self):
        config = {
            'axis': self.axis,
            'epsilon': self.epsilon,
            'center': self.center,
            'scale': self.scale,
            'beta_initializer': initializers.serialize(self.beta_initializer),
            'gamma_initializer': initializers.serialize(self.gamma_initializer),
            'beta_regularizer': regularizers.serialize(self.beta_regularizer),
            'gamma_regularizer': regularizers.serialize(self.gamma_regularizer),
            'beta_constraint': constraints.serialize(self.beta_constraint),
            'gamma_constraint': constraints.serialize(self.gamma_constraint)
        }
        base_config = super(InstanceNormalization, self).get_config()

        return dict(list(base_config.items()) + list(config.items()))

In [42]:
# Patch GAN discriminator

"""In a traditional GAN, the discriminator typically outputs a single value indicating whether the entire input image is real or fake. In contrast, a PatchGAN discriminates at the patch level, providing a spatially detailed assessment of the realism of different regions in the input image.
The PatchGAN discriminator produces a grid or map of output values, where each value corresponds to the realism of a local patch in the input image. This approach is especially useful for tasks like image-to-image translation, where the goal is to generate realistic high-resolution output images."""

def create_discriminator(image_shape):
    init = RandomNormal(stddev=0.02)
    in_image = Input(shape=image_shape)
    d = Conv2D(64, (4, 4), strides=(2, 2), padding='same', kernel_initializer=init)(in_image)
    d = LeakyReLU(alpha=0.2)(d)
    d = Conv2D(128, (4, 4), strides=(2, 2), padding='same', kernel_initializer=init)(d)
    d = InstanceNormalization(axis=-1)(d)
    d = LeakyReLU(alpha=0.2)(d)
    d = Conv2D(256, (4, 4), strides=(2, 2), padding='same', kernel_initializer=init)(d)
    d = InstanceNormalization(axis=-1)(d)
    d = LeakyReLU(alpha=0.2)(d)
    
    d = tf.image.resize(d, size=(165, 165), method=tf.image.ResizeMethod.BICUBIC)

    
    patch_out = Conv2D(1, (4, 4), padding='same', kernel_initializer=init)(d)
    model = Model(in_image, patch_out)
    model.compile(loss='mse', optimizer=Adam(lr=0.0002), loss_weights=[0.5])
    """The loss for the discriminator is weighted by 50% for each model update.
    This slows down changes to the discriminator relative to the generator model during training."""
    return model


# def create_discriminator(image_shape):
#     init = tf.keras.initializers.HeNormal() #RandomNormal(stddev=0.02)
#     in_image = Input(shape=image_shape)
#     d = Conv1D(64, 4, strides=2, padding='same', kernel_initializer=init)(in_image)
#     d = LeakyReLU(alpha=0.2)(d)
#     d = Conv1D(128, 4, strides=2, padding='same', kernel_initializer=init)(d)
#     d = InstanceNormalization(axis=-1)(d)
#     d = LeakyReLU(alpha=0.2)(d)
#     d = Conv1D(256, 4, strides=2, padding='same', kernel_initializer=init)(d)
#     d = InstanceNormalization(axis=-1)(d)
#     d = LeakyReLU(alpha=0.2)(d)
#     patch_out = Conv1D(1, 4, padding='same', kernel_initializer=init)(d)
#     model = Model(in_image, patch_out)
#     model.compile(loss='mse', optimizer=Adam(lr=0.0002), loss_weights=[0.5])
#     """The loss for the discriminator is weighted by 50% for each model update.
#     This slows down changes to the discriminator relative to the generator model during training."""
#     return model
    

In [43]:

# ResNet block
def resnet_block(n_filters, input_layer):
    init = RandomNormal(stddev=0.02)
    g = Conv2D(n_filters, (3, 3), padding='same', kernel_initializer=init)(input_layer)
    g = InstanceNormalization(axis=-1)(g)
    g = Activation('relu')(g)
    g = Conv2D(n_filters, (3, 3), padding='same', kernel_initializer=init)(g)
    g = InstanceNormalization(axis=-1)(g)
    g = Concatenate()([g, input_layer])
    return g

# Generator model
# def create_generator(image_shape, n_resnet=3):
#     init = tf.keras.initializers.HeNormal() #RandomNormal(stddev=0.02)
#     in_image = Input(shape=image_shape)
#     g = Conv2D(64, (7, 7), padding='same', kernel_initializer=init)(in_image)
#     g = InstanceNormalization(axis=-1)(g)
#     g = Activation('relu')(g)
# #     g = Conv2D(128, (3, 3), strides=(2, 2), padding='same', kernel_initializer=init)(g)
#     g = Conv2D(128, (3, 3), padding='same', kernel_initializer=init)(g)
#     g = InstanceNormalization(axis=-1)(g)
#     g = Activation('relu')(g)
#     g = Conv2D(256, (3, 3), strides=(2, 2), padding='same', kernel_initializer=init)(g)
#     g = InstanceNormalization(axis=-1)(g)
#     g = Activation('relu')(g)
#     for _ in range(n_resnet):
#         g = resnet_block(256, g)
#     g = Conv2DTranspose(128, (3, 3), strides=(2, 2), padding='same', kernel_initializer=init)(g)
#     g = InstanceNormalization(axis=-1)(g)
#     g = Activation('relu')(g)
#     g = Conv2DTranspose(64, (3, 3), strides=(2, 2), padding='same', kernel_initializer=init)(g)
#     g = InstanceNormalization(axis=-1)(g)
#     g = Activation('relu')(g)
#     g = Conv2D(3, (7, 7), padding='same', kernel_initializer=init)(g)
#     g = InstanceNormalization(axis=-1)(g)
#     out_image = Activation('tanh')(g)
#     # using tanh as it has bigger gradient, helps againt vanishing gradients
#     model = Model(in_image, out_image)
#     return model

def create_generator(image_shape, n_resnet=3):
    init = RandomNormal(stddev=0.02)
    in_image = Input(shape=image_shape)
    g = Conv2D(64, (7, 7), padding='same', kernel_initializer=init)(in_image)
    g = InstanceNormalization(axis=-1)(g)
    g = Activation('relu')(g)
#     g = Conv2D(128, (3, 3), strides=(2, 2), padding='same', kernel_initializer=init)(g)
    g = Conv2D(128, (3, 3), padding='same', kernel_initializer=init)(g)
    g = InstanceNormalization(axis=-1)(g)
    g = Activation('relu')(g)
    g = Conv2D(256, (3, 3), padding='same', kernel_initializer=init)(g)
    g = InstanceNormalization(axis=-1)(g)
    g = Activation('relu')(g)
#     for _ in range(n_resnet):
#         g = resnet_block(256, g)
    g = Conv2DTranspose(128, (3, 3), padding='same', kernel_initializer=init)(g)
    g = InstanceNormalization(axis=-1)(g)
    g = Activation('relu')(g)
    g = Conv2DTranspose(64, (3, 3), padding='same', kernel_initializer=init)(g)
    g = InstanceNormalization(axis=-1)(g)
    g = Activation('relu')(g)
    g = Conv2D(1, (7, 7), padding='same', kernel_initializer=init)(g)
    g = InstanceNormalization(axis=-1)(g)
    out_image = Activation('tanh')(g)
    # using tanh as it has bigger gradient, helps againt vanishing gradients
    model = Model(in_image, out_image)
    return model



In [44]:
# Composite model for updating generators by adversarial and cycle loss
def define_composite_model(g_model_1, d_model, g_model_2, image_shape):
    g_model_1.trainable = True
    d_model.trainable = False
    g_model_2.trainable = False
    input_gen = Input(shape=image_shape)
    gen1_out = g_model_1(input_gen)
    output_d = d_model(gen1_out)
    input_id = Input(shape=image_shape)
    output_id = g_model_1(input_id)
    output_f = g_model_2(gen1_out)
    gen2_out = g_model_2(input_id)
    output_b = g_model_1(gen2_out)
    model = Model([input_gen, input_id], [output_d, output_id, output_f, output_b])
    model.compile(loss=['mse', 'mae', 'mae', 'mae'], loss_weights=[1, 5, 10, 10], optimizer=Adam(lr=0.0002))
    return model


### Data Loading and prepration

In [45]:


# Generate real samples
def generate_real_samples(df, n_samples, patch_shape):
    ix = randint(0, df.shape[0], n_samples)
    X = df[ix]
    y = ones((n_samples, patch_shape, patch_shape, 1))
    return X, y

# Generate fake samples
def generate_fake_samples(g_model, df, patch_shape):
    X = g_model.predict(df)
    y = zeros((len(X), patch_shape, patch_shape, 1))
    return X, y

# Save generator models
def save_models(step, g_model_AtoB, g_model_BtoA):
    g_model_AtoB.save('trained/g_model_AtoB.h5')
    g_model_BtoA.save('trained/g_model_BtoA.h5')

In [46]:
genre_path_1 = '../dataset/JC_C/train'
JC_C_names = sorted([os.path.join(genre_path_1, f) for f in os.listdir(genre_path_1) if os.path.isfile(os.path.join(genre_path_1, f))])

In [47]:
genre_path_2 = '../dataset/JC_J/train'
JC_J_names = sorted([os.path.join(genre_path_2, f) for f in os.listdir(genre_path_2) if os.path.isfile(os.path.join(genre_path_2, f))])

In [48]:
JC_C_np = np.array([np.load(x) for x in JC_C_names[:10000]])
JC_C_np.shape

(10000, 64, 84, 1)

In [49]:
JC_J_np = np.array([np.load(x) for x in JC_J_names[:10000]])
JC_J_np.shape

(10000, 64, 84, 1)

In [50]:
# A = JC_C_np.reshape((10000, 1, 64, 84, 1))
# B = JC_J_np.reshape((10000, 1, 64, 84, 1))

In [51]:
# A.shape, B.shape

In [52]:
A_B_dataset = np.array([JC_C_np, JC_J_np])

In [53]:
A_B_dataset.shape

(2, 10000, 64, 84, 1)

In [54]:
# datay = np.concatenate(( jzz_s, ctry_s), axis=0)
# train_df = [datay, cls_s]

In [55]:
# load data as train_df
# train_df = preprocessing(train_df)

In [56]:
# # train_df = np.array(train_df)
# train_df.shape
# train_df = np.expand_dims(train_df,axis = -1)
# # train_df = train_df.reshape(2,50,1314,1025,1).transpose(0,1,4,2,3)
# train_df.shape

In [57]:
# train_df.dtype

### Model Training

In [58]:
def update_image_pool(pool, images, max_size=50):
    selected = []
    for image in images:
        if len(pool) < max_size:
            pool.append(image)
            selected.append(image)
        elif random() < 0.5:
            selected.append(image)
        else:
            ix = randint(0, len(pool))
            selected.append(pool[ix])
            pool[ix] = image
    return np.asarray(selected)

In [64]:
def train(d_model_A, d_model_B, g_model_AtoB, g_model_BtoA, c_model_AtoB, c_model_BtoA, epochs=1):
    n_epochs, n_batch = epochs, 1
    n_patch = d_model_A.output_shape[1]
    trainA, trainB = A_B_dataset
    poolA, poolB = [], []
    bat_per_epo = len(trainA) // n_batch
    n_steps = bat_per_epo * n_epochs
    
    for i in range(n_steps):
        X_realA, y_realA = generate_real_samples(trainA, n_batch, n_patch)
        X_realB, y_realB = generate_real_samples(trainB, n_batch, n_patch)
        X_fakeA, y_fakeA = generate_fake_samples(g_model_BtoA, X_realB, n_patch)
        X_fakeB, y_fakeB = generate_fake_samples(g_model_AtoB, X_realA, n_patch)

        X_fakeA = update_image_pool(poolA, X_fakeA)
        X_fakeB = update_image_pool(poolB, X_fakeB)
        
        g_loss2, _, _, _, _ = c_model_BtoA.train_on_batch([X_realB, X_realA], [y_realA, X_realA, X_realB, X_realA])
        dA_loss1 = d_model_A.train_on_batch(X_realA, y_realA)
        dA_loss2 = d_model_A.train_on_batch(X_fakeA, y_fakeA)
        
        g_loss1, _, _, _, _ = c_model_AtoB.train_on_batch([X_realA, X_realB], [y_realB, X_realB, X_realA, X_realB])
        dB_loss1 = d_model_B.train_on_batch(X_realB, y_realB)
        dB_loss2 = d_model_B.train_on_batch(X_fakeB, y_fakeB)
        
        print('Iteration>%d, dA[%.3f,%.3f] dB[%.3f,%.3f] g[%.3f,%.3f]' % (i+1, dA_loss1, dA_loss2, dB_loss1, dB_loss2, g_loss1, g_loss2))
        
        if (i+1) % (bat_per_epo * 10) == 0:
            save_models(i, g_model_AtoB, g_model_BtoA)

In [65]:
image_shape = A_B_dataset[0].shape[1:]

g_model_AtoB = create_generator(image_shape)
g_model_BtoA = create_generator(image_shape)

d_model_A = create_discriminator(image_shape)
d_model_B = create_discriminator(image_shape)

c_model_AtoB = define_composite_model(g_model_AtoB, d_model_B, g_model_BtoA, image_shape)
c_model_BtoA = define_composite_model(g_model_BtoA, d_model_A, g_model_AtoB, image_shape)


/Users/adityaramachandra/miniconda3/lib/python3.10/site-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


In [ ]:
#call for train fuction
train(d_model_A, d_model_B, g_model_AtoB, g_model_BtoA, c_model_AtoB, c_model_BtoA, epochs=5)

2023-12-07 20:57:21.317890: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 181ms/step


2023-12-07 20:57:23.871620: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node Adam/AssignAddVariableOp.


### Performance testing (manual)

In [ ]:
## Load a single sample, name is real_A
# convert it: -1 to 1

In [ ]:
# load model if not loaded

# model_AtoB = load_model('trained/g_model_AtoB.h5')
# model_BtoA = load_model('trained/g_model_BtoA.h5')

In [ ]:
B_hat = model_AtoB.predict(real_A)
A_hat = model_BtoA.predict(B_hat)

In [ ]:
#convert predictions to audio